## MYOCARDIAL RESISTANCE ALGORITHM

   This file contains experiments done for the actual algorithm that estimates myocardial resistance.

        This file includes: Data pre-processins and machine learning algorithms.
        
Author: Aldair M Silva, University of Sheffield, INSIGNEO SUMMER PLACEMENT 2019

## DATA IMPORT SECTION

    In this section, libraries and data files are imported to the program.
    
### Import libraries

In [1]:
import statsmodels.formula.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

import scipy as sp
from scipy import random, stats
from scipy.stats import spearmanr

import seaborn as sb
import seaborn as sns

import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
import sklearn.model_selection as LogReg
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVR
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

import os
import numpy as np
from math import isnan
from pylab import rcParams
import matplotlib.pyplot as plt

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import math
import sys
np.random.seed(0)
rcParams["figure.figsize"] = 10,5
sb.set_style("whitegrid")

### Import data from file

In [2]:
data_path = "../Desktop/INSIGNEO-internship-2019---Coronary-resistance-master/DATASET" # location of spreadsheet
data_file_name = "MR_test_dataset_T.xlsx" # Speadsheet name
data_file = data_path + data_file_name
rawdata = pd.read_excel(data_file) # Layout data in pandas dataframe

FileNotFoundError: [Errno 2] No such file or directory: '../Desktop/INSIGNEO-internship-2019---Coronary-resistance-master/DATASETMR_test_dataset_T.xlsx'

## DATA PRE-PROCESSING SECTION
    
    In this section, data will be arranged in DataFrames, divided into subgroups, scaled, and set up in the right formats 
      and types to be used efficiently later in the program.
      
### Specify data subsets

In [ ]:
# Set up labels
all_cols = ['Case No','Patient no','Myocardial resistance','Vessel','LAD','LCX_Int','RCA','Om_Dx_PDA','Inlet diameter','Outlet diameter','Minimum diameter','Myocardial Jeopardy Index','Duke jeopardy score','Age','Male','Height','Weight','Heart rate','Ever smoked','Previous MI','Hyperlipidaemia','Diabetes','PVD','COPD','CCS score','Frailty score','Mean Pa','No of Anti-anginals','No of anti-hypertensives','Beta blocker','ACE inhibitor_ARB','Nitrate','eGFR','Haematocrit','ECG QRS duration','ECG QRS axis','ECG QTC interval','ECG LVH voltage criteria','Angio SYNTAX score','Bifurcation lesion','CVFTSV','New york risk score']

# Create subset for myocardial resistance
MR = rawdata[all_cols[2]]

# Relevant features
mFFR = ['Computed vFFR'] #Does not exist in new dataset
MR_cols = ['Myocardial resistance']
Vessels = ['Vessel']
Useful_cols = ['Inlet diameter','Outlet diameter','Minimum diameter','Myocardial Jeopardy Index','Duke jeopardy score','Age','Male','Height','Weight','Heart rate','Ever smoked','Previous MI','Hyperlipidaemia','Diabetes','PVD','COPD','CCS score','Frailty score','Mean Pa','No of Anti-anginals','No of anti-hypertensives','Beta blocker','ACE inhibitor_ARB','Nitrate','eGFR','Haematocrit','ECG QRS duration','ECG QRS axis','ECG QTC interval','ECG LVH voltage criteria','Angio SYNTAX score','Bifurcation lesion','CVFTSV','New york risk score']

DIn = rawdata[Useful_cols[0]]
DOut = rawdata[Useful_cols[1]]
DMin = rawdata[Useful_cols[2]]
MJeo_index = rawdata[Useful_cols[3]]
DJeo_index = rawdata[Useful_cols[4]]
Age = rawdata[Useful_cols[5]]
Male = rawdata[Useful_cols[6]]
Height = rawdata[Useful_cols[7]]
Weight = rawdata[Useful_cols[8]]
Heart_rate = rawdata[Useful_cols[9]]
Ever_smoked = rawdata[Useful_cols[10]]
Previous_MI = rawdata[Useful_cols[11]]
Hyperlipidaemia = rawdata[Useful_cols[12]]
Diabetes = rawdata[Useful_cols[13]]
PVD = rawdata[Useful_cols[14]]
COPD = rawdata[Useful_cols[15]]
CCS_score = rawdata[Useful_cols[16]]
Frailty_score = rawdata[Useful_cols[17]]
Mean_Pa = rawdata[Useful_cols[18]]
No_of_Anti_anginals = rawdata[Useful_cols[19]]
No_of_anti_hypertensives = rawdata[Useful_cols[20]]
Beta_blocker = rawdata[Useful_cols[21]]
ACE_inhibitor_ARB = rawdata[Useful_cols[22]]
Nitrate = rawdata[Useful_cols[23]]
eGFR = rawdata[Useful_cols[24]]
Haematocrit = rawdata[Useful_cols[25]]
ECG_QRS_duration = rawdata[Useful_cols[26]]
ECG_QRS_axis = rawdata[Useful_cols[27]]
ECG_QTC_interval = rawdata[Useful_cols[28]]
ECG_LVH_voltage_criteria = rawdata[Useful_cols[29]]
Angio_SYNTAX_score = rawdata[Useful_cols[30]]
Bifurcation_lesion = rawdata[Useful_cols[31]]
CVFTSV = rawdata[Useful_cols[32]]
NY_risk_score = rawdata[Useful_cols[33]]
    
# Create a dataframe from the dataset
Patient_ID = np.array(rawdata.iloc[:,1])
patient_list = Patient_ID.reshape(-1,1)

df = pd.DataFrame(rawdata,columns=all_cols)

Age2 = rawdata.iloc[:,-3]
vFFR = rawdata.iloc[:,-2]
MR2 = rawdata.iloc[:,-1]

#df.reset_index()

## Scaling data

In this section the data is going to be rescaled for computational purposes.
* At the moment, only the myocardial resistance will be rescaled by 10^(-9)

In [ ]:
def normalise(data):
    max_point = data.max()
    min_point = data.min()
    norm = (data-min_point)/(max_point-min_point)
    return norm

MR_norm = normalise(MR2)

### Dealing with missing data

There will be 2 methods of dealing with this kind of situations:

* Row removal
    * Where the whole row will be discarted
* Mean override
    * Where the value with be substituted with the mean of that particular category(i.e. mean of all LAD/RCA)

### Method 1: Row removal

This causes 11 patients to be removed from the dataset

In [ ]:
# Nan finder function
F_col = (MR,DIn,DOut,DMin,MJeo_index,DJeo_index,Age,Male,Height,Weight,Heart_rate,Ever_smoked,Previous_MI,Hyperlipidaemia,Diabetes,PVD,COPD,CCS_score,Frailty_score,Mean_Pa,No_of_Anti_anginals,No_of_anti_hypertensives,Beta_blocker,ACE_inhibitor_ARB,Nitrate,eGFR,Haematocrit,ECG_QRS_duration,ECG_QRS_axis,ECG_QTC_interval,ECG_LVH_voltage_criteria,Angio_SYNTAX_score,Bifurcation_lesion,CVFTSV,NY_risk_score) # Identify feature 
    
nans1 = df[F_col[0].isnull()] # Find nans in data frame
nans2 = df[F_col[1].isnull()] 
nans3 = df[F_col[2].isnull()] 
nans4 = df[F_col[3].isnull()] 
nans5 = df[F_col[4].isnull()] 
nans6 = df[F_col[5].isnull()] 
nans7 = df[F_col[6].isnull()] 
nans8 = df[F_col[7].isnull()] 
nans9 = df[F_col[8].isnull()] 
nans10 = df[F_col[9].isnull()] 
nans11 = df[F_col[10].isnull()] 
nans12 = df[F_col[11].isnull()] 
nans13 = df[F_col[12].isnull()] 
nans14 = df[F_col[13].isnull()]
nans15 = df[F_col[14].isnull()]
nans16 = df[F_col[15].isnull()]
nans17 = df[F_col[16].isnull()]
nans18 = df[F_col[17].isnull()]
nans19 = df[F_col[18].isnull()]
nans20 = df[F_col[19].isnull()]
nans21 = df[F_col[20].isnull()]
nans22 = df[F_col[21].isnull()]
nans23 = df[F_col[22].isnull()]
nans24 = df[F_col[23].isnull()]
nans25 = df[F_col[24].isnull()]
nans26 = df[F_col[25].isnull()]
nans27 = df[F_col[26].isnull()]
nans28 = df[F_col[27].isnull()]
nans29 = df[F_col[28].isnull()]
nans30 = df[F_col[29].isnull()]
nans31 = df[F_col[30].isnull()]
nans32 = df[F_col[31].isnull()]
nans33 = df[F_col[32].isnull()]
nans34 = df[F_col[33].isnull()]


all_nans = nans1 + nans2 + nans3 + nans4 + nans5 + nans6 + nans7 + nans8 + nans9 + nans10 + nans11 + nans12 + nans13 + nans14 + nans15 + nans16 + nans17 + nans18 + nans19 + nans20 + nans21 + nans22 + nans23 + nans24 + nans25 + nans26 + nans27 + nans28 + nans29 + nans30 + nans31 + nans32 + nans33 + nans34

NAN_i = np.array(all_nans.index)

def filter_nans(data,nan_data):
    filtered_data = np.delete(np.array(data),[nan_data],axis=0)
    return filtered_data

def list_data(data):
    data_list = list(data)
    return data_list

MR_filtered = filter_nans(F_col[0],NAN_i) # Remove nan columns
DIn_filtered = filter_nans(F_col[1],NAN_i)
DOut_filtered = filter_nans(F_col[2],NAN_i)
DMin_filtered = filter_nans(F_col[3],NAN_i)
MJeo_index_filtered = filter_nans(F_col[4],NAN_i)
DJeo_index_filtered = filter_nans(F_col[5],NAN_i)
Age_filtered = filter_nans(F_col[6],NAN_i)
Male_filtered = filter_nans(F_col[7],NAN_i)
Height_filtered = filter_nans(F_col[8],NAN_i)
Weight_filtered = filter_nans(F_col[9],NAN_i)
Heart_rate_filtered = filter_nans(F_col[10],NAN_i)
Ever_smoked_filtered = filter_nans(F_col[11],NAN_i)
Previous_MI_filtered = filter_nans(F_col[12],NAN_i)
Hyperlipidaemia_filtered = filter_nans(F_col[13],NAN_i)
Diabetes_filtered = filter_nans(F_col[14],NAN_i)
PVD_filtered = filter_nans(F_col[15],NAN_i)
COPD_filtered = filter_nans(F_col[16],NAN_i)
CCS_score_filtered = filter_nans(F_col[17],NAN_i)
Frailty_score_filtered = filter_nans(F_col[18],NAN_i)
Mean_Pa_filtered = filter_nans(F_col[19],NAN_i)
No_of_Anti_anginals_filtered = filter_nans(F_col[20],NAN_i)
No_of_anti_hypertensives_filtered = filter_nans(F_col[21],NAN_i)
Beta_blocker_filtered = filter_nans(F_col[22],NAN_i)
ACE_inhibitor_ARB_filtered = filter_nans(F_col[23],NAN_i)
Nitrate_filtered = filter_nans(F_col[24],NAN_i)
eGFR_filtered = filter_nans(F_col[25],NAN_i)
Haematocrit_filtered = filter_nans(F_col[26],NAN_i)
ECG_QRS_duration_filtered = filter_nans(F_col[27],NAN_i)
ECG_QRS_axis_filtered = filter_nans(F_col[28],NAN_i)
ECG_QTC_interval_filtered = filter_nans(F_col[29],NAN_i)
ECG_LVH_voltage_criteria_filtered = filter_nans(F_col[30],NAN_i)
Angio_SYNTAX_score_filtered = filter_nans(F_col[31],NAN_i)
Bifurcation_lesion_filtered = filter_nans(F_col[32],NAN_i)
CVFTSV_filtered = filter_nans(F_col[33],NAN_i)
NY_risk_score_filtered = filter_nans(F_col[34],NAN_i)
MR2_filtered = filter_nans(MR_norm,NAN_i)

Patients_filtered = filter_nans(patient_list,NAN_i)

MR2_list = list_data(MR2_filtered)
MR_list = list_data(MR_filtered) # Make list from filtered data
DIn_list = list_data(DIn_filtered)
DOut_list = list_data(DOut_filtered)
DMin_list = list_data(DMin_filtered)
MJeo_index_list = list_data(MJeo_index_filtered)
DJeo_index_list = list_data(DJeo_index_filtered)
Age_list = list_data(Age_filtered)
Male_list = list_data(Male_filtered)
Height_list = list_data(Height_filtered)
Weight_list = list_data(Weight_filtered)
Heart_rate_list = list_data(Heart_rate_filtered)
Ever_smoked_list = list_data(Ever_smoked_filtered)
Previous_MI_list = list_data(Previous_MI_filtered)
Hyperlipidaemia_list = list_data(Hyperlipidaemia_filtered)
Diabetes_list = list_data(Diabetes_filtered)
PVD_list = list_data(PVD_filtered)
COPD_list = list_data(COPD_filtered)
CCS_score_list = list_data(CCS_score_filtered)
Frailty_score_list = list_data(Frailty_score_filtered)
Mean_Pa_list = list_data(Mean_Pa_filtered)
No_of_Anti_anginals_list = list_data(No_of_Anti_anginals_filtered)
No_of_anti_hypertensives_list = list_data(No_of_anti_hypertensives_filtered)
Beta_blocker_list = list_data(Beta_blocker_filtered)
ACE_inhibitor_ARB_list = list_data(ACE_inhibitor_ARB_filtered)
Nitrate_list = list_data(Nitrate_filtered)
eGFR_list = list_data(eGFR_filtered)
Haematocrit_list = list_data(Haematocrit_filtered)
ECG_QRS_duration_list = list_data(ECG_QRS_duration_filtered)
ECG_QRS_axis_list = list_data(ECG_QRS_axis_filtered)
ECG_QTC_interval_list = list_data(ECG_QTC_interval_filtered)
ECG_LVH_voltage_criteria_list = list_data(ECG_LVH_voltage_criteria_filtered)
Angio_SYNTAX_score_list = list_data(Angio_SYNTAX_score_filtered)
Bifurcation_lesion_list = list_data(Bifurcation_lesion_filtered)
CVFTSV_list = list_data(CVFTSV_filtered)
NY_risk_score_list = list_data(NY_risk_score_filtered)

MR2_df = pd.DataFrame(MR2_list)
MR_df = pd.DataFrame(MR_list)

Model_list = ['DIn_list','DOut_list','DMin_list','MJeo_index_list','DJeo_index_list','Age_list','Male_list','Height_list','Weight_list','Heart_rate_list','Ever_smoked_list','Previous_MI_list','Hyperlipidaemia_list','Diabetes_list','PVD_list','COPD_list','CCS_score_list','Frailty_score_list','Mean_Pa_list','No_of_Anti_anginals_list','No_of_anti_hypertensives_list','Beta_blocker_list','ACE_inhibitor_ARB_list','Nitrate_list','eGFR_list','Haematocrit_list','ECG_QRS_duration_list','ECG_QRS_axis_list','ECG_QTC_interval_list','ECG_LVH_voltage_criteria_list','Angio_SYNTAX_score_list','Bifurcation_lesion_list','CVFTSV_list','NY_risk_score_list']

In [ ]:
#Model 1
def buildModel(model, vPredict,listVariables):
    modelSyntax_part2 = ' + '.join(listVariables)
    if (model == 'ols'):
        modelSyntax = 'sm.ols(formula = \'' + vPredict + ' ~ ' + modelSyntax_part2 + '\', data=MR_df).fit()'
    return modelSyntax

a = buildModel('ols', 'MR_list', Model_list)

print(a)
model1 = eval(a)
#print(model1.params['Intercept'])
model1.summary()

In [ ]:
plt.hist(model1.resid)
plt.title('Histogram of Residue in Multiple-linear regression')
plt.xlabel('Model Residue')
plt.ylabel('Frequency')

In [ ]:
all_pars = (DIn_filtered,DOut_filtered,DMin_filtered,MJeo_index_filtered,DJeo_index_filtered,Age_filtered,Male_filtered,Height_filtered,Weight_filtered,Heart_rate_filtered,Ever_smoked_filtered,Previous_MI_filtered,Hyperlipidaemia_filtered,Diabetes_filtered,PVD_filtered,COPD_filtered,CCS_score_filtered,Frailty_score_filtered,Mean_Pa_filtered,No_of_Anti_anginals_filtered,No_of_anti_hypertensives_filtered,Beta_blocker_filtered,ACE_inhibitor_ARB_filtered,Nitrate_filtered,eGFR_filtered,Haematocrit_filtered,ECG_QRS_duration_filtered,ECG_QRS_axis_filtered,ECG_QTC_interval_filtered,ECG_LVH_voltage_criteria_filtered,Angio_SYNTAX_score_filtered,Bifurcation_lesion_filtered,CVFTSV_filtered,NY_risk_score_filtered)

x1r = all_pars[0].reshape(-1,1)
x2r = all_pars[1].reshape(-1,1)
x3r = all_pars[2].reshape(-1,1)
x4r = all_pars[3].reshape(-1,1)
x5r = all_pars[4].reshape(-1,1)
x6r = all_pars[5].reshape(-1,1)
x7r = all_pars[6].reshape(-1,1)
x8r = all_pars[7].reshape(-1,1)
x9r = all_pars[8].reshape(-1,1)
x10r = all_pars[9].reshape(-1,1)
x11r = all_pars[10].reshape(-1,1)
x12r = all_pars[11].reshape(-1,1)
x13r = all_pars[12].reshape(-1,1)
x14r = all_pars[13].reshape(-1,1)
x15r = all_pars[14].reshape(-1,1)
x16r = all_pars[15].reshape(-1,1)
x17r = all_pars[16].reshape(-1,1)
x18r = all_pars[17].reshape(-1,1)
x19r = all_pars[18].reshape(-1,1)
x20r = all_pars[19].reshape(-1,1)
x21r = all_pars[20].reshape(-1,1)
x22r = all_pars[21].reshape(-1,1)
x23r = all_pars[22].reshape(-1,1)
x24r = all_pars[23].reshape(-1,1)
x25r = all_pars[24].reshape(-1,1)
x26r = all_pars[25].reshape(-1,1)
x27r = all_pars[26].reshape(-1,1)
x28r = all_pars[27].reshape(-1,1)
x29r = all_pars[28].reshape(-1,1)
x30r = all_pars[29].reshape(-1,1)
x31r = all_pars[30].reshape(-1,1)
x32r = all_pars[31].reshape(-1,1)
x33r = all_pars[32].reshape(-1,1)
x34r = all_pars[33].reshape(-1,1)

data = (x1r,x2r,x3r,x4r,x5r,x6r,x7r,x8r,x9r,x10r,x11r,x12r,x13r,x14r,x15r,x16r,x17r,x18r,x19r,x20r,x21r,x22r,x23r,x24r,x25r,x26r,x27r,x28r,x29r,x30r,x31r,x32r,x33r,x34r)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, MR_filtered, test_size=0.2, random_state=0)

print('Train shape: ', x_train.shape, y_train.shape)
print('Test shape: ', x_test.shape, y_test.shape)

clf = SVR(gamma='scale', C=18.0, epsilon=0.2)
clf.fit(x_train, y_train) 
#clf.predict(x_train) # predicted values
print('R^2 of prediction: ',clf.score(x_train,y_train))